<p style="text-align: center;"> <span style="color:firebrick"> <font size="5"> <b> USC Marshall School of Business </b> </font> </p> </span> 

<p style="text-align: center;"> <b> <font font size="5"> DSO 560 - Homework 1 </p> </b></font>

<p style="text-align: center;"> <b> Spring 2021 </b> </p>

In [1]:
# import packages
import pandas as pd
import seaborn as sns
import re

# Question 1
A manager in the marketing department wants to find out the most frequently used words in positive reviews (five stars) and negative reviews (one star) in order to determine what occasion the toys are purchased for (Christmas, birthdays, and anniversaries.). He would like your opinion on which gift occasions (Christmas, birthdays, or anniversaries) tend to have the most positive reviews to focus marketing budget on those days.

In [2]:
# open the good comments and poor comments
good_tf = open("good_amazon_toy_reviews.txt", "r")
good_lines = good_tf.readlines()

poor_tf = open("poor_amazon_toy_reviews.txt", "r")
poor_lines = poor_tf.readlines()

print(f"There are {len(good_lines)} lines in the good comments.")

There are 102217 lines in the good comments.


In [3]:
# define a function to count the word
def word_count(lines, word):
    count = 0
    for i in range(len(lines)):
        count += len(re.findall(word, lines[i])) 
    return count

In [4]:
# run the function to get the word count for each gift occasions
C_count = word_count(good_lines, r'\b(Christmas|christmas)\b')
B_count = word_count(good_lines, r'\b(Birthday|birthday)\b')
A_count = word_count(good_lines, r'\b(Anniversary|anniversary)\b')

print(f'The word "christmas" has occurred in positive reviews for {C_count} times.')
print(f'The word "birthday" has occurred in positive reviews for {B_count} times.')
print(f'The word "anniversary" has occurred in positive reviews for {A_count} times.')

The word "christmas" has occurred in positive reviews for 1187 times.
The word "birthday" has occurred in positive reviews for 3911 times.
The word "anniversary" has occurred in positive reviews for 52 times.


From the above analysis, we find that birthday occurred most in positive reviews. Thus, we should focus marketing budget on birthdays.

# Question 2
There are malformed characters in the review text. For instance, notice the `&#34;` - these are examples of incorrectly decoded [HTML encodings](https://krypted.com/utilities/html-encoding-reference/).

```
"amazing quality first of all, these cards are amazing proxies (but don't try to use em in &#34;official duels&#34; unless a judge is okay with it, if you have the real thing to show) and look amazing in your binder!"
```

Please clean up all instances of these incorrect decodings.

In [5]:
# count how many '&#\d\d;' and '<br />' are in the good reviews and poor reviews
comma_count = word_count(good_lines, r'(&#\d\d;)')
linebreak_count = word_count(good_lines, r'(<br />)|\n')
print(f'The word "&#\d\d;" has occurred in positive reviews for {comma_count} times.')
print(f'The word "<br />" has occurred in positive reviews for {linebreak_count} times.')

comma_count = word_count(poor_lines, r'(&#\d\d;)')
linebreak_count = word_count(poor_lines, r'(<br />)|\n')
print(f'The word "&#\d\d;" has occurred in negative reviews for {comma_count} times.')
print(f'The word "<br />" has occurred in negative reviews for {linebreak_count} times.')

The word "&#\d\d;" has occurred in positive reviews for 8358 times.
The word "<br />" has occurred in positive reviews for 124233 times.
The word "&#\d\d;" has occurred in negative reviews for 1619 times.
The word "<br />" has occurred in negative reviews for 14849 times.


In [6]:
# define a function to replace the words
def word_replace(lines, word, re_word):
    for i in range(len(lines)):
        lines[i] = re.sub(word, re_word, lines[i])
    return lines

In [7]:
# run the function to replace the malformed characters
good_lines = word_replace(good_lines, r"(&#\d\d;)", "'")
good_lines = word_replace(good_lines, r"(<br />)|\n", "")
poor_lines = word_replace(poor_lines, r"(&#\d\d;)", "'")
poor_lines = word_replace(poor_lines, r"(<br />)|\n", "")

In [8]:
# count how many '&#\d\d;' and '<br />' are in the good reviews and poor reviews after cleaning up
comma_count = word_count(good_lines, r'(&#\d\d;)')
linebreak_count = word_count(good_lines, r'(<br />)|\n')
print(f'The word "&#\d\d;" has occurred in positive reviews for {comma_count} times.')
print(f'The word "<br />" has occurred in positive reviews for {linebreak_count} times.')

comma_count = word_count(poor_lines, r'(&#\d\d;)')
linebreak_count = word_count(poor_lines, r'(<br />)|\n')
print(f'The word "&#\d\d;" has occurred in negative reviews for {comma_count} times.')
print(f'The word "<br />" has occurred in negative reviews for {linebreak_count} times.')

The word "&#\d\d;" has occurred in positive reviews for 0 times.
The word "<br />" has occurred in positive reviews for 0 times.
The word "&#\d\d;" has occurred in negative reviews for 0 times.
The word "<br />" has occurred in negative reviews for 0 times.


# Question 3
One of your product managers suspects that toys purchased for male recipients (husbands, sons, etc.) tend to be much more likely to be reviewed poorly. She would like to see some data points confirming or rejecting her hypothesis.

In [9]:
# count how many recipients occurred in the good reviews and poor reviews
male_count_poor = word_count(poor_lines, r'\b(son|Son|sons|Sons|husband|Husband|Father|father)\b')
female_count_poor = word_count(poor_lines, r'\b(daughter|Daughter|daughters|Daughters|wife|Wife|Mother|mother)\b')
male_count_good = word_count(good_lines, r'\b(son|Son|sons|Sons|husband|Husband|Father|father)\b')
female_count_good = word_count(good_lines, r'\b(daughter|Daughter|daughters|Daughters|wife|Wife|Mother|mother)\b')
prob_male = male_count_poor/(male_count_poor + male_count_good)
prob_female = female_count_poor/(female_count_poor + female_count_good)
print('The probability of Male Recipients giving poor reviews:', prob_male)
print('The probability of Female Recipients giving poor reviews:', prob_female)

The probability of Male Recipients giving poor reviews: 0.08494505494505494
The probability of Female Recipients giving poor reviews: 0.05843263061411549


By comparison, male recipients tend to give more poor reviews than female recipients by 3%. Thus, we think toys purchased for male recipients tend to be much more likely to be reviewed poorly.

# Question 4
Use regular expressions to parse out all references to recipients and gift occassions, and account for the possibility that people may spell words "son" / "children" / "Christmas" as both singular and plural, upper or lower-cased.

In [10]:
# define a function to parse out words to pandas tables
def parse_word(lines, word):
    word_dict = {}
    for i in range(len(lines)):
        words = re.findall(word, lines[i], re.I)
        for j in words:
            if j not in word_dict.keys():
                word_dict[j] = 1
            else:
                word_dict[j] += 1
    word_df = pd.DataFrame(list(word_dict.items()),columns = ['word','count']) 
    word_df = word_df.sort_values(by=["word"]).reset_index(drop = True)
    return word_df

In [11]:
# pares out "son" in the good reviews
parse_word(good_lines, r'\b(son|sons)\b')

,word,count
0,SON,37
1,SONS,1
2,Son,249
3,Sons,3
4,son,6803
5,sons,377


In [12]:
# parse out "children" in the good reviews
parse_word(good_lines, r'\b(child|children)\b')

,word,count
0,CHILD,14
1,CHILDREN,16
2,Child,48
3,Children,69
4,child,2158
5,children,1779


In [13]:
# parse out "Christmas" in the good reviews
parse_word(good_lines, r'\b(Christmas|X-?mas)\b')

,word,count
0,CHRISTMAS,11
1,Christmas,1103
2,X-Mas,3
3,X-mas,6
4,XMAS,2
5,Xmas,54
6,christmas,80
7,x-mas,3
8,xmas,21


In [14]:
# parse out other words for references to recipients and gift occassions in the good reviews
parse_word(good_lines, r'\b(daughter|daughters|mother|father|husband|wife|birthday|bithdays|anniversary|anniversaries)\b')

,word,count
0,Anniversary,7
1,BIRTHDAY,24
2,Birthday,207
3,DAUGHTER,15
4,Daughter,328
5,Daughters,13
6,Father,16
7,HUSBAND,3
8,Husband,30
9,Mother,12


# Question 6 (Optional)
Create a simple text file that contains 2-3 lines at most describing yourself, your background, and interests. It must contain at least 1 emoji and 4-5 international characters (non-ASCII). Make sure to properly encode the file so that I can open it in UTF-8 to read. I must be able to read all characters properly. Attach it to your submission.

In [4]:
import re
# get the function of change utf-8
def get_binary_for_char(char: str, encoding="utf-8") -> str:
    """
    Encodes a character using the desired encoding into its corresponding hex, then converts the
    hex code into binary, formatted with tab spaces between byte marks.
    """

    hex_code = char.encode(encoding).hex()
    code_point = hex(ord(char))[2:].upper()

    binary: str = f"{int(hex_code, 16):08b}"

    byte_list: List[str] = re.findall('[01]{8}', binary)
    formatted_binary: str = "\t".join(byte_list)  # for variable length encoding, tab space between byte marks.
    print(f"{char} (U+{code_point.zfill(4)}, hex:{hex_code}) - {encoding}: {formatted_binary}")
    return formatted_binary

def get_binary(text: str, encoding="utf-8"):
    return " ".join([get_binary_for_char(char, encoding) for char in text])

In [6]:
# run the function
get_binary('My name is ___. I am currently a USC student. I love reading 📚.', encoding="utf-8")

M (U+004D, hex:4d) - utf-8: 01001101
y (U+0079, hex:79) - utf-8: 01111001
  (U+0020, hex:20) - utf-8: 00100000
n (U+006E, hex:6e) - utf-8: 01101110
a (U+0061, hex:61) - utf-8: 01100001
m (U+006D, hex:6d) - utf-8: 01101101
e (U+0065, hex:65) - utf-8: 01100101
  (U+0020, hex:20) - utf-8: 00100000
i (U+0069, hex:69) - utf-8: 01101001
s (U+0073, hex:73) - utf-8: 01110011
  (U+0020, hex:20) - utf-8: 00100000
_ (U+005F, hex:5f) - utf-8: 01011111
_ (U+005F, hex:5f) - utf-8: 01011111
_ (U+005F, hex:5f) - utf-8: 01011111
. (U+002E, hex:2e) - utf-8: 00101110
  (U+0020, hex:20) - utf-8: 00100000
I (U+0049, hex:49) - utf-8: 01001001
  (U+0020, hex:20) - utf-8: 00100000
a (U+0061, hex:61) - utf-8: 01100001
m (U+006D, hex:6d) - utf-8: 01101101
  (U+0020, hex:20) - utf-8: 00100000
c (U+0063, hex:63) - utf-8: 01100011
u (U+0075, hex:75) - utf-8: 01110101
r (U+0072, hex:72) - utf-8: 01110010
r (U+0072, hex:72) - utf-8: 01110010
e (U+0065, hex:65) - utf-8: 01100101
n (U+006E, hex:6e) - utf-8: 01101110
t

'01001101 01111001 00100000 01101110 01100001 01101101 01100101 00100000 01101001 01110011 00100000 01011111 01011111 01011111 00101110 00100000 01001001 00100000 01100001 01101101 00100000 01100011 01110101 01110010 01110010 01100101 01101110 01110100 01101100 01111001 00100000 01100001 00100000 01010101 01010011 01000011 00100000 01110011 01110100 01110101 01100100 01100101 01101110 01110100 00101110 00100000 01001001 00100000 01101100 01101111 01110110 01100101 00100000 01110010 01100101 01100001 01100100 01101001 01101110 01100111 00100000 11110000\t10011111\t10010011\t10011010 00101110'